## Key metrics summary

In [0]:
spark.sql("""
SELECT 
    'Total Loans' as metric,
    COUNT(*) as value
FROM msme_risk_analytics.silver_loan_default
UNION ALL
SELECT 'Default Rate %', ROUND(AVG(Status)*100, 2)
FROM msme_risk_analytics.silver_loan_default
UNION ALL
SELECT 'Avg Loan Amount', ROUND(AVG(loan_amount), 2)
FROM msme_risk_analytics.silver_loan_default
UNION ALL
SELECT 'Total GST (₹Cr)', ROUND(SUM(fy_total), 2)
FROM msme_risk_analytics.silver_gst_unified
""").show()


+---------------+----------+
|         metric|     value|
+---------------+----------+
| Default Rate %|     35.62|
|Avg Loan Amount| 239911.07|
|Total GST (₹Cr)|7635910.72|
|    Total Loans|   16552.0|
+---------------+----------+



## High-risk segments

In [0]:
spark.sql("""
SELECT loan_purpose, Region,
       COUNT(*) as loans,
       ROUND(AVG(Status)*100, 2) as default_rate,
       ROUND(AVG(loan_amount), 0) as avg_loan
FROM msme_risk_analytics.silver_loan_default
GROUP BY loan_purpose, Region
HAVING COUNT(*) > 100
ORDER BY default_rate DESC
LIMIT 10
""").show()

+------------+-------+-----+------------+--------+
|loan_purpose| Region|loans|default_rate|avg_loan|
+------------+-------+-----+------------+--------+
|          p4|  south|  857|       57.99|242381.0|
|          p4|central|  215|       54.42|241384.0|
|          p4|  North|  960|       47.29|257521.0|
|          p2|  south|  175|       46.29| 88671.0|
|          p2|  North|  216|       43.06| 85389.0|
|          p3|central|  698|       41.83|226973.0|
|          p3|  south| 3734|       35.65|234149.0|
|          p1|  south| 1674|       33.63|259487.0|
|          p3|  North| 3943|       31.17|224725.0|
|          p1|central|  563|       30.37|270425.0|
+------------+-------+-----+------------+--------+



## Top 10 GST states (Z targets)

In [0]:
spark.sql("""
SELECT state_name,
       ROUND(SUM(fy_total), 2) as total_6years,
       ROUND(AVG(fy_total), 2) as avg_annual
FROM msme_risk_analytics.silver_gst_unified
GROUP BY state_name
ORDER BY total_6years DESC
LIMIT 10
""").show(truncate=False)

+-------------+------------+----------+
|state_name   |total_6years|avg_annual|
+-------------+------------+----------+
|Maharashtra  |1585953.34  |264325.56 |
|Karnataka    |714531.3    |119088.55 |
|Gujarat      |641467.75   |106911.29 |
|Tamil Nadu   |603920.84   |100653.47 |
|Haryana      |515584.08   |85930.68  |
|Uttar Pradesh|513877.86   |85646.31  |
|Delhi        |338693.91   |56448.98  |
|West Bengal  |322739.07   |53789.84  |
|Telangana    |299731.21   |49955.2   |
|Odisha       |279238.42   |46539.74  |
+-------------+------------+----------+



## Credit score vs default

In [0]:
spark.sql("""
SELECT credit_category,
       COUNT(*) as loans,
       ROUND(AVG(Status)*100, 2) as default_rate
FROM msme_risk_analytics.silver_ml_features
GROUP BY credit_category
ORDER BY default_rate DESC
""").show()

+---------------+-----+------------+
|credit_category|loans|default_rate|
+---------------+-----+------------+
|      Excellent| 6258|       36.26|
|           Fair| 4147|       35.74|
|           Poor| 2115|       35.46|
|           Good| 4032|       34.57|
+---------------+-----+------------+

